In [ ]:
import pandas as pd
import numpy as np
from glob import glob
import json

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
from tqdm.notebook import tqdm as tqdm

import plotly.graph_objs as go
from PIL import Image

import sys
sys.path.append('../input/indoor-locationnavigation-2021/indoor-location-competition-20-master')
from io_f import read_data_file
from main import calibrate_magnetic_wifi_ibeacon_to_position
from main import extract_wifi_count

In [ ]:
TRAIN_DIR = '../input/indoor-location-navigation/train'
META_DIR = '../input/indoor-location-navigation/metadata'

In [ ]:
def get_all_building_id():
    return sorted(list(map(lambda x: x.split('/')[-1], glob(f'{META_DIR}/*'))))

def get_all_building_floor():
    all_building_floor = {}
    all_building_id = get_all_building_id()
    for building_id in all_building_id:
        all_building_floor[building_id] = sorted(list(map(lambda x: x.split('/')[-1], glob(f'{META_DIR}/{building_id}/*'))))
    return all_building_floor

def get_all_building_floor_df():
    all_building_floor = get_all_building_floor()
    df = pd.DataFrame.from_dict(all_building_floor, orient='index')
    df = df.unstack().dropna().sort_index(level=1).reset_index(drop=False).drop(columns=['level_0'])
    df = df.rename(columns={'level_1': 'building_id', 0: 'floor'})
    return df

buiding_df = get_all_building_floor_df()
buiding_df

In [ ]:
def plot_all_wifi_count_in_floor(building_id, floor_id):
    trace_paths = glob(f'{TRAIN_DIR}/{building_id}/{floor_id}/*')

    heat_positions = []
    heat_values = []
    for trace_path in tqdm(trace_paths):
        try:
            mwi_datas = calibrate_magnetic_wifi_ibeacon_to_position([trace_path])

            wifi_counts = extract_wifi_count(mwi_datas)
            heat_positions_sub = np.array(list(wifi_counts.keys()))
            heat_values_sub = np.array(list(wifi_counts.values()))
            # filter out positions that no wifi detected
            mask = heat_values_sub != 0
            heat_positions_sub = heat_positions_sub[mask]
            heat_values_sub = heat_values_sub[mask]

            heat_positions.extend(list(heat_positions_sub))
            heat_values.extend(heat_values_sub)
        except Exception as e:
            print (f'** ERROR in {trace_path}: {e}')


    fig = go.Figure()

    floor_plan_filename = f'{META_DIR}/{building_id}/{floor_id}/floor_image.png'

    # Prepare width_meter & height_meter
    ### (taken from the .json file)
    json_plan_filename = f'{META_DIR}/{building_id}/{floor_id}/floor_info.json'
    with open(json_plan_filename) as json_file:
        json_data = json.load(json_file)

    width_meter = json_data["map_info"]["width"]
    height_meter = json_data["map_info"]["height"]

    position = np.array(heat_positions)
    value = heat_values
    floor_plan_filename = floor_plan_filename
    width_meter = width_meter
    height_meter = height_meter
    colorbar_title="colorbar"
    title=f'{building_id}_{floor_id}'
    show=False
    g_size=700
    colorscale="Rainbow"

    # add heat map
    fig.add_trace(
        go.Scatter(x=position[:, 0],
                   y=position[:, 1],
                   mode='markers',
                   marker=dict(size=7,
                               color=value,
                               colorbar=dict(title=colorbar_title),
                               colorscale=colorscale),
                   text=value,
                   name=title))

    # add floor plan
    floor_plan = Image.open(floor_plan_filename)
    fig.update_layout(images=[
        go.layout.Image(
            source=floor_plan,
            xref="x",
            yref="y",
            x=0,
            y=height_meter,
            sizex=width_meter,
            sizey=height_meter,
            sizing="contain",
            opacity=1,
            layer="below",
        )
    ])

    # configure
    fig.update_xaxes(autorange=False, range=[0, width_meter])
    fig.update_yaxes(autorange=False, range=[0, height_meter], scaleanchor="x", scaleratio=1)
    fig.update_layout(
        title=go.layout.Title(
            text=title or "No title.",
            xref="paper",
            x=0,
        ),
        autosize=True,
        width=g_size,
        height=200 + g_size * height_meter / width_meter,
        template="plotly_white",
    )
    
    fig.show()

In [ ]:
for building_id, floor_id in buiding_df[['building_id', 'floor']].values[:10]:
    plot_all_wifi_count_in_floor(building_id, floor_id)